# Recommender Systems - Data Mining Lab 2

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

### Data loading

In [ ]:
train_file_path = 'data/lab2_train.csv' 
test_file_path = 'data/lab2_test.csv'

train_data = pd.read_csv(train_file_path, delimiter=',')
test_data = pd.read_csv(test_file_path, delimiter=',')


In [ ]:
train_data.dropna(axis=0, inplace=True)

like_train_data = train_data[train_data.is_like == True]
match_train_data = train_data[train_data.is_match == True]
dislike_train_data = train_data[train_data.is_like == False]

liked_user_count = like_train_data["user_to_id"].nunique()
user_count = like_train_data["user_from_id"].nunique()
match_count = match_train_data["user_from_id"].nunique()

number_given_likes = like_train_data["user_from_id"]
likes_given_counts = number_given_likes.value_counts()
dislikes_given_counts = dislike_train_data["user_from_id"].value_counts()

plt.figure(figsize=(15, 8))
plt.bar(likes_given_counts.index, likes_given_counts.values, color='blue')

plt.xlabel('User IDs')
plt.ylabel('Number of given Likes')
plt.title('Distribution of Number of given Likes per User ID')
plt.xticks([])  # Hides x axis labels due to the large number of ids
plt.show()

number_of_likes = like_train_data["user_to_id"]
received_counts = number_of_likes.value_counts()

plt.figure(figsize=(15, 8))
plt.bar(received_counts.index, received_counts.values, color='blue')



In [ ]:
given_and_received_likes_per_id = pd.concat([likes_given_counts, received_counts], axis=1)
given_and_received_likes_per_id.fillna(0, inplace=True)
given_and_received_likes_per_id.columns = ["given likes", "received likes"]

given_and_received_dislikes_per_id = pd.concat([dislikes_given_counts, received_counts], axis=1)
given_and_received_dislikes_per_id.fillna(0, inplace=True)
given_and_received_dislikes_per_id.columns = ["given dislikes", "received likes"]

given_and_received_likes_per_id.plot.scatter(x="given likes", y="received likes", alpha=0.5, figsize=(15,8))

plt.xlabel("number of given likes")
plt.ylabel("number of received likes")
plt.title("Number of given likes vs number of received likes")
plt.show()

given_and_received_dislikes_per_id.plot.scatter(x="given dislikes", y="received likes", alpha=0.5, figsize=(15,8))

plt.xlabel("number of given dislikes")
plt.ylabel("number of received likes")
plt.title("Number of given dislikes vs Number of received likes")
plt.show()

activity = likes_given_counts.add(dislikes_given_counts, fill_value=0)
activity_per_id = pd.concat([activity, received_counts], axis=1)
activity_per_id.fillna(0, inplace=True)
activity_per_id.columns = ["activity", "received likes"]

activity_per_id.plot.scatter(x= "activity", y="received likes", alpha=0.5, figsize=(15,8))
plt.xlabel("number of given likes or dislikes")
plt.ylabel("number of received likes")
plt.title("Activity of user vs Number of received likes")
plt.show()

## Familiarization
### Important properties of the data
There are only 1707 user who have one like or more of the 2107 users who gave likes. Of all of these users there are only 415 matches. Of these matches there are 345 unique users who receive a match. So there is only a small group of users who receive more than one match. And a small number of matches on the total of 76392 likes and dislikes.

The Data is therefore very sparse. Of these 76392 likes and dislikes there are only 12637 likes. This is 16,5% likes. Of these likes there are only 415 matches. Thus only 3,3% mutual likes. This is sparse data to work with. You see in the visualization that there is some correlation between the number of received likes and the number of given likes. You see in the visualization above that the more likes a person received correlates with the less likes a user gives. Since that user gets a lot of likes the user gets critical for liking another user. Thus it also holds for when a user doesn't receive many likes than that user has is less critical of other users and gives more likes.


### Types of people

- Someone who receives a lot of likes but likes almost no one. User 2 receives 45 likes but likes none of them. Therefore not leading to a match

- Someone who receives and gives about only dislikes. For example user 28 is very active on the platform but gives about only dislikes and receives only dislikes. Therefore not leading to a match

- Someone who gives a lot of likes and receives only dislikes. For example user 18 gives 26 likes and receives no likes back and has thus not a match.

- Someone who receives a lot of likes and gives a lot of likes. For example user 36 gives a lot of likes and receives a lot of likes and eventualy leading to a match.

## Non-negative matrix factorization

In [ ]:
user_data = train_data["user_from_id"].value_counts()

valid_users = user_data[user_data >= 5].index
train_data = train_data[train_data["user_from_id"].isin(valid_users)]

train_data = train_data.drop_duplicates()

def nmf(X, mask, hyper_param, n_components: int, max_iter: int=1000, tol: float=1e-3):
  """
  Decomposes the original sparse matrix X into two matrices W and H. 
  """
  # Initialize W and H with random non-negative values
  W = np.random.rand(X.shape[0], n_components)
  H = np.random.rand(n_components, X.shape[1])

  E = np.sum(((X - W @ H)**2) * mask)
  newE = 0.
  i = 0

  x_masked = X * mask
  while E - newE > tol and i < max_iter: 
    E = np.sum(((X - W @ H)**2) * mask)

    nominatorH = W.T @ x_masked
    denominatorH = (W.T @ ((W @ H) * mask) + 1e-9)
    H = H * nominatorH / denominatorH

    nominatorW = (x_masked @ H.T)
    denominatorW = (((W @ H) * mask) @ H.T + 1e-9)
    W = W * nominatorW / denominatorW

    newE = np.sum(((X - W @ H)**2) * mask)
    i += 1

  print(E)
  print(i)
  return W, H

In [ ]:
x = train_data.to_numpy()[:,[0,1,3]].astype(float)
y = train_data.to_numpy()[:, 2]
y = np.where(y == False, 0., 1.)
x[:,2] = np.where(x[:,2] == False, 0., 1.)
x[:,:2] = x[:,:2] / 5000.
y = y.astype(float)
x = x.astype(float)


def split_data():
  x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.001, random_state=42)

  mask = np.ones((len(x), 4))
  mask[len(x_train):, 3:] = 0

  train_matrix = np.hstack((np.vstack((x_train, x_test)), np.vstack((y_train.reshape(-1, 1), y_test.reshape(-1, 1)))))
  return train_matrix, y_train, mask, len(x_train)

def eval(X, M, components, train_length):
  ys = []

  W, H = nmf(X, M, 100., components)
  predicted_matrix = W @ H
  return predicted_matrix[:train_length, 3:]

def calculate():
  for j in range(35, 36, 1):
    plt.figure(figsize=(15,8))
    train_matrix, y_test, mask, train_length = split_data()
    results_gotten = eval(train_matrix, mask, j, train_length)
    results_actual = y_test
    plt.scatter(results_actual, results_gotten, label=f"{j} components", alpha=0.5)
    plt.xlabel("real score")
    plt.ylabel("predicted score")
    plt.title("real score vs predicted score")
    plt.legend()
    plt.show()

calculate()


## Mask
As can be seen in the nmf method, we make use of masked_x to improve the speed. And we perform element-wise multiplication in the error function and the cumulative update function.

## Components
We have done extensive testing, and we have seen the best performance for 35 components. This variable can be changed in the code, to compare the results. The pre exisiting range can be filled in for more ease during visualization.

## Pre-processing
We have removed all users that have liked less than 5 users, or have less than 5 likes. Additionally, we remove the duplicates from the dataframe.

## Normalization
Since the features(id's), have significantly larger values than the classification values. The range of 0-3500 compared to 0-1. We therefore chose to scale the feature value down by 5000.

## Recommendation threshold
Since it seems like the values are located at 0 & 1, we want to place our threshold at 0.5 for maximum margin.

Using this pipeline, we have first split the input data into a train and verification dataset. We then train this data by use of a mask. We then plot the data that we have masked out, and see that we can correctly predict the data. 

# Distance-based pipeline

In [ ]:
from scipy.sparse import csr_matrix
# pre-processing the training data
def create_binary_matrix(train_data):
    user_from_ids = train_data['user_from_id'].unique()
    user_to_ids = train_data['user_to_id'].unique()
    matrix = np.zeros((len(user_from_ids), len(user_to_ids)), dtype=int)

    for _, row in train_data.iterrows():
        user_from_index = np.where(user_from_ids == row['user_from_id'])[0][0]
        user_to_index = np.where(user_to_ids == row['user_to_id'])[0][0]
        # Assigning weights: 2 for match, 1 for like
        matrix[user_from_index, user_to_index] = 2 if row['is_match'] else (1 if row['is_like'] else 0)


    return csr_matrix(matrix), user_from_ids, user_to_ids

matrix, user_from_ids, user_to_ids = create_binary_matrix(train_data)

# Using the Min Hash function to get the signatures using random hash values
def min_hash(matrix, num_hash_functions=4):
    num_rows, num_cols = matrix.shape
    signatures = np.full((num_hash_functions, num_cols), np.inf)

    for r in range(num_rows):
        hash_values = np.random.permutation(num_rows)[:num_hash_functions]
        for c in matrix[r].nonzero()[1]:
            signatures[:, c] = np.minimum(signatures[:, c], hash_values)

    return signatures

signatures = min_hash(matrix)

# Finding the similarity in signatures using Jaccard distance
def jaccard_similarity(sig_col1, sig_col2):
    return np.sum(sig_col1 == sig_col2) / len(sig_col1)

# Find nearest neighbors for a user_id
def nearest_neighbors(signatures, target_col, k=5):
    similarities = []
    for i in range(signatures.shape[1]):
        if i != target_col:
            similarity = jaccard_similarity(signatures[:, target_col], signatures[:, i])
            similarities.append((i, similarity))
    similarities.sort(key=lambda x: x[1], reverse=True)
    return similarities[:k]

# Find nearest neighbors for a user_id of choice
neighbors = nearest_neighbors(signatures, 36)
print("Nearest neighbors:", neighbors)

# Report on Distance based Recommender System

## Preprocessing

### Data transformation
- **Input Data**: The raw dataset consists of user interactions, including the data values `user_from_id`, `user_to_id`, `is_like`, and `is_match`.
- **Binary Matrix Creation**: The data is transformed into a binary matrix. Rows represent users, and columns represent items. Cells in the matrix indicate the presence (1) or absence (0) of an interaction.

### Weighting scheme (Enhanced Preprocessing)
- **Importance Differentiation**: The system differentiates between 'like' and 'match'. 'Match' is given higher importance with a weight of 2, and 'like' is assigned a weight of 1.
- **Matrix Representation**: This weighting scheme is incorporated into the binary matrix, enhancing the traditional binary representation to reflect the strength of interactions.

## Min Hashing

- **Hash Function Generation**: A set of hash functions is generated. The number of functions (k) is a tunable parameter. After tuning we chose to use here for 4 randomized hash functions. To give a balanced result.
- **Signature Matrix Creation**: Each user is hashed using these functions to create a min-hash signature, reducing dimensionality while preserving similarity.

## Nearest neighbor search

- **Jaccard Distance Calculation**: The Jaccard distance between min hash signatures is used to quantify similarity between the users.
- **Nearest Neighbors Identification**: For each user the system identifies the nearest neighbors. These are the users with the most similar min-hash signatures. We have chosen to return the top 5 of these nearest neighboars. In the case of Breeze these 5 users can be shown on a certain day. 5 is a feasable and not overwhelming number. Thus keeping the goals and process of the app intact.

## Post-processing

- **Aggregation for Prediction**: For a new user-item pair, the system estimates the likelihood of interaction (like or match) based on the information aggregated from nearest neighbors.
- **Recommendation Criteria**: Recommendations are based on the interactions of nearest neighbors, prioritizing items with higher aggregated scores.
- **Result Delivery**: The system outputs a list of the top five recommended users for each user, sorted by the likelihood of a like and match.

## Analyse result

You see in the result of the distance based recommender system when using it that there are users who receive neighbours with high likelihood and there are users who receive neighbours with lower likelihood. This is a normal result for this kind of program. Because this depends on various criteria. If a certain user doesn't get any likes and doesn't give any likes. Their likelihood of liking a certain neigbour and receiving a match is very low. This you see back in the result. 
